In [53]:
import geopandas as gpd

import numpy as np
import pandas as pd
from pandas import IndexSlice as idx
import tensorflow as tf
import sys
import os
import glob

from functools import partial



code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/perturbations/'
sys.path.append(code_dir)
code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/diff_bpr'
sys.path.append(code_dir)
from top_k import top_k_idx
#from make_datasets import make_data
from bpr_model import PerturbedBPRLinearModel, PerturbedBPRMLPModel
from make_datasets import make_data


code_dir = '/cluster/home/kheuto01/code/opioid-overdose-models/'
sys.path.append(code_dir)
from zinf_gp.metrics import normcdf, fixed_top_X



from perturbations import perturbed
from bpr import bpr_variable_k_no_ties

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from bpr import bpr_variable_k_no_ties
from sklearn.ensemble import HistGradientBoostingRegressor

In [41]:
data_path='/cluster/tufts/hugheslab/datasets/NSF_OD/results_20220606_update/clean_annual_tract/'
seed = 360
lookback_years=5
learning_rate=0.01
add_spacetime=True
add_svi=True

time_window = lookback_years
timesteps_per_year = 1
first_train_eval_year = 2013
last_train_eval_year = 2017
batch_dim_size = (last_train_eval_year - first_train_eval_year + 1)*timesteps_per_year
validation_year = 2018
first_test_year = 2019
last_test_year = 2020


tf.random.set_seed(seed)


timestep_col = 'timestep'
geography_col = 'geoid'
outcome_col = 'deaths'

x_idx_cols = [geography_col, 'lat', 'lon', timestep_col,
              'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
              'svi_pctile', 'year',
              'neighbor_t', 'deaths']
y_idx_cols = [geography_col, timestep_col, outcome_col]
features_only = ['deaths']

if add_spacetime:
    features_only += ['lat', 'lon', timestep_col]
if add_svi:
    features_only += ['theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc', 'svi_pctile']


#data_gdf = gpd.read_file(data_path)

multiindexed_gdf = data_gdf.set_index(['geoid', 'timestep'])
multiindexed_gdf['timestep'] = multiindexed_gdf.index.get_level_values('timestep')
num_geoids = len(data_gdf['geoid'].unique())

train_shape = (num_geoids, time_window, len(features_only))

train_x_BSF_flat, train_y_BS = make_data(multiindexed_gdf, first_train_eval_year, last_train_eval_year,
                                                  time_window, features_only, train_shape, pred_lag=timesteps_per_year)

valid_x_BSF_flat, valid_y_BS = make_data(multiindexed_gdf, validation_year, validation_year,
                                         time_window, features_only, train_shape, pred_lag=timesteps_per_year)

test_x_BSF_flat, test_y_BS = make_data(multiindexed_gdf, first_test_year, last_test_year,
                                       time_window, features_only, train_shape, pred_lag=timesteps_per_year)

norm_layer = tf.keras.layers.Normalization()
norm_layer.adapt(train_x_BSF_flat)
train_x_BSF_flat = norm_layer(train_x_BSF_flat)
valid_x_BSF_flat = norm_layer(valid_x_BSF_flat)
test_x_BSF_flat = norm_layer(test_x_BSF_flat)

top_100_idx_func = partial(top_k_idx, k=100)
from zinf_gp.metrics import normcdf, fixed_top_X

In [42]:
linear_poisson = sklearn.linear_model.PoissonRegressor()

In [43]:
features_only

['deaths',
 'lat',
 'lon',
 'timestep',
 'theme_1_pc',
 'theme_2_pc',
 'theme_3_pc',
 'theme_4_pc',
 'svi_pctile']

In [44]:
train_x_long = tf.reshape(train_x_BSF_flat,((lookback_years*1620),lookback_years*len(features_only)))
train_y_long = tf.reshape(train_y_BS,(lookback_years*1620,1))

In [45]:
reg = linear_poisson.fit(train_x_long, tf.squeeze(train_y_long))

In [46]:
pred_linear_2019 = reg.predict(test_x_BSF_flat[0])
bpr_variable_k_no_ties(test_y_BS[0], pred_linear_2019,k=100)

<tf.Tensor: shape=(), dtype=float32, numpy=0.58760685>

In [47]:
pred_linear_2020 = reg.predict(test_x_BSF_flat[1])
bpr_variable_k_no_ties(test_y_BS[1], pred_linear_2020,k=100)

<tf.Tensor: shape=(), dtype=float32, numpy=0.61723447>

In [48]:
true_deaths = multiindexed_gdf.loc[idx[:,18],:].drop(columns='timestep').reset_index().set_index('geoid')['deaths']
deaths_index = true_deaths.index.values

In [49]:

this_2019 = multiindexed_gdf.loc[idx[:,19],:].drop(columns='timestep').reset_index().set_index('geoid')['deaths']
this_2020 = multiindexed_gdf.loc[idx[:,20],:].drop(columns='timestep').reset_index().set_index('geoid')['deaths']

In [50]:
%%time
bprs_2019=[]
bprs_2020=[]
rng = np.random.default_rng(seed=360)
removed_tracts = 250
num_tracts = test_x_BSF_flat.shape[1]
num_sampled = num_tracts - removed_tracts
K=100

for _ in range(100):
    sampled_indicies = rng.choice(range(num_tracts), size=num_sampled, replace=False)
    
    bprs_2019.append(fixed_top_X_new(this_2019[sampled_indicies],
                            pd.Series(pred_linear_2019[sampled_indicies], index = this_2019[sampled_indicies].index),
                            X=K)[-1])
    bprs_2020.append(fixed_top_X_new(this_2020[sampled_indicies],
                            pd.Series(pred_linear_2020[sampled_indicies], index = this_2019[sampled_indicies].index),
                            X=K)[-1])

CPU times: user 58.2 s, sys: 142 ms, total: 58.4 s
Wall time: 58.2 s


In [51]:
avg_bprs = (np.array(bprs_2019)+np.array(bprs_2020))/2
np.percentile(avg_bprs,2.5), np.mean(avg_bprs), np.percentile(avg_bprs, 97.5)

(0.5877864678656495, 0.6091694879172294, 0.6285353158816639)

In [14]:
pred_linear_2019

array([0.7119827 , 0.90811805, 0.70009379, ..., 0.63995345, 0.63995345,
       0.72337451])

In [50]:
(0.58760685+0.61723447)/2

0.6024206599999999

In [51]:
(0.5982906+0.6032064)/2

0.6007485

In [52]:
np.mean([mean_absolute_error(test_y_BS[0], pred_linear_2019),
mean_absolute_error(test_y_BS[1], pred_linear_2020)])

1.052748407663083

In [53]:
np.mean([sqrt(mean_squared_error(test_y_BS[0], pred_linear_2019)),
sqrt(mean_squared_error(test_y_BS[1], pred_linear_2020))])

1.3809252248671406

In [56]:
from sklearn.ensemble import HistGradientBoostingRegressor
max_valid=0
for depth in [3,6,9]:
    for leaf in [2,5,10]:
        for sampl in [50,100,400]:
            hist_poisson =   HistGradientBoostingRegressor(loss="poisson", max_iter=10000, max_depth=depth, max_leaf_nodes=leaf,
                                                           l2_regularization=0.1, min_samples_leaf=sampl )
            hist_reg = hist_poisson.fit(train_x_long, tf.squeeze(train_y_long))
            
            valid_fit = hist_reg.predict(valid_x_BSF_flat[0])
            new_valid = bpr_variable_k_no_ties(valid_y_BS[0], valid_fit,k=100)
            
            if new_valid>max_valid:
                print(f'{depth}, {leaf}, {sampl}')
                max_valid=new_valid
                print(max_valid)

3, 2, 50
tf.Tensor(0.5782881, shape=(), dtype=float32)


KeyboardInterrupt: 

In [54]:
hist_poisson =   HistGradientBoostingRegressor(loss="poisson", max_iter=10000, max_depth=3, max_leaf_nodes=2,
                                                           l2_regularization=1, min_samples_leaf=100 )
hist_reg = hist_poisson.fit(train_x_long, tf.squeeze(train_y_long))

In [55]:
pred_hist_2019 = hist_reg.predict(test_x_BSF_flat[0])
bpr_variable_k_no_ties(test_y_BS[0], pred_hist_2019,k=100)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5534188>

In [56]:
pred_hist_2020 = hist_reg.predict(test_x_BSF_flat[1])
bpr_variable_k_no_ties(test_y_BS[1], pred_hist_2020,k=100)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5751503>

In [57]:
%%time
bprs_2019=[]
bprs_2020=[]
rng = np.random.default_rng(seed=360)
removed_tracts = 250
num_tracts = test_x_BSF_flat.shape[1]
num_sampled = num_tracts - removed_tracts
K=100

for _ in range(100):
    sampled_indicies = rng.choice(range(num_tracts), size=num_sampled, replace=False)
    
    bprs_2019.append(fixed_top_X_new(this_2019[sampled_indicies],
                            pd.Series(pred_hist_2019[sampled_indicies], index = this_2019[sampled_indicies].index),
                            X=K)[-1])
    bprs_2020.append(fixed_top_X_new(this_2020[sampled_indicies],
                            pd.Series(pred_hist_2020[sampled_indicies], index = this_2019[sampled_indicies].index),
                            X=K)[-1])

CPU times: user 58.4 s, sys: 143 ms, total: 58.5 s
Wall time: 58.3 s


In [58]:
avg_bprs = (np.array(bprs_2019)+np.array(bprs_2020))/2
np.percentile(avg_bprs,2.5), np.mean(avg_bprs), np.percentile(avg_bprs, 97.5)

(0.5542042284628933, 0.5744520979737323, 0.5967444282797825)

In [60]:
train_fit = hist_reg.predict(train_x_BSF_flat[0])
bpr_variable_k_no_ties(train_y_BS[0], train_fit,k=100)

<tf.Tensor: shape=(), dtype=float32, numpy=0.48398575>

In [61]:
valid_fit = hist_reg.predict(valid_x_BSF_flat[0])
bpr_variable_k_no_ties(valid_y_BS[0], valid_fit,k=100)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5803758>

In [62]:
(0.5751503+0.5534188)/2

0.56428455

In [68]:
np.mean([sqrt(mean_squared_error(test_y_BS[0], pred_hist_2019)),
sqrt(mean_squared_error(test_y_BS[1], pred_hist_2020))])

1.20849009336153

In [70]:
np.mean([mean_absolute_error(test_y_BS[0], pred_hist_2019),
mean_absolute_error(test_y_BS[1], pred_hist_2020)])

0.9019008909597088

In [217]:
bpr_variable_k_no_ties(valid_y_BS[0], valid_fit*0+1,k=100)

<tf.Tensor: shape=(), dtype=float32, numpy=0.217119>

In [220]:
perturbed_top_100 = perturbed(top_100_idx_func,
                              num_samples=5000,
                              sigma=0.05,
                              noise='normal',
                              batched=True)


In [225]:
fixed_top_X( multiindexed_gdf[(multiindexed_gdf['year']==2019)]['deaths'], 
            multiindexed_gdf[(multiindexed_gdf['year']==2019)]['deaths']*0,X=100)

(468.0, 0.0, 352.975, 0.24577991452991454)

In [226]:
fixed_top_X( multiindexed_gdf[(multiindexed_gdf['year']==2020)]['deaths'], 
            multiindexed_gdf[(multiindexed_gdf['year']==2020)]['deaths']*0,X=100)

(499.0, 0.0, 378.734, 0.24101402805611225)

In [227]:
(0.24577991452991454+0.24101402805611225)/2

0.2433969712930134

In [12]:
tf.reduce_sum(tf.sort(test_y_BS[0])[-100:])

<tf.Tensor: shape=(), dtype=float32, numpy=468.0>

In [13]:
(468+499)/2

483.5

In [15]:
483.5*0.004

1.934

In [29]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt


In [25]:
def fixed_top_X_new(true_qtr_val, pred_qtr_val, X=10):
    top_X_predicted = pred_qtr_val.sort_values(ascending=False).iloc[:X]
    top_X_true = true_qtr_val.sort_values(ascending=False).iloc[:X]

    undisputed_top_predicted = top_X_predicted[top_X_predicted > top_X_predicted.min()]
    num_tied_spots = X - len(undisputed_top_predicted)
    undisputed_top_true = top_X_true[top_X_true > top_X_true.min()]
    num_true_ties = X - len(undisputed_top_true)

    tied_top_predicted = pred_qtr_val[pred_qtr_val == top_X_predicted.min()]
    tied_top_true = true_qtr_val[true_qtr_val == top_X_true.min()]

    error_in_top_true_ties = np.abs(tied_top_true - pred_qtr_val[tied_top_true.index]).sort_values(ascending=True)
    error_in_top_pred_ties = np.abs(true_qtr_val[tied_top_predicted.index] - tied_top_predicted).sort_values(
        ascending=True)
    top_true_tied_geoids = error_in_top_true_ties.iloc[:num_true_ties].index
    top_pred_tied_geoids = error_in_top_pred_ties.iloc[:num_tied_spots].index

    best_possible_top_true_geoids = pd.Index.union(undisputed_top_true.index, top_true_tied_geoids)
    best_possible_top_pred_geoids = pd.Index.union(undisputed_top_predicted.index, top_pred_tied_geoids)

    # True values of GEOIDS with highest actual deaths. If ties, finds tied locations that match preds best
    best_possible_true = true_qtr_val[best_possible_top_true_geoids]
    best_possible_pred = true_qtr_val[best_possible_top_pred_geoids]
    assert (len(best_possible_true) == X)
    assert (len(best_possible_pred) == X)

    best_possible_absolute = np.abs(best_possible_true.sum() - best_possible_pred.sum())
    best_possible_ratio = np.abs(best_possible_pred).sum() / np.abs(best_possible_true).sum()

    bootstrapped_tied_indices = np.random.choice(tied_top_predicted.index, (1000, num_tied_spots))
    this_idx = undisputed_top_predicted.index.values
    bootstrapped_all_indices = [np.concatenate((this_idx,bootstrap_index)) for bootstrap_index in bootstrapped_tied_indices]
    #bootstrapped_all_indices = [pd.Index.union(undisputed_top_predicted.index,
    #                                           bootstrap_index) for bootstrap_index in bootstrapped_tied_indices]
    
    denominator =  top_X_true.sum()
    numerators = [true_qtr_val[indicies].sum() for indicies in bootstrapped_all_indices] 

    bootstrapped_absolute = np.mean([np.abs(denominator - numerator)
                                     for numerator in numerators])
    bootstrapped_ratio = np.mean([numerator / denominator
                                  for numerator in numerators])

    return best_possible_absolute, best_possible_ratio, bootstrapped_absolute, bootstrapped_ratio